In [1]:
import tensorflow as tf 
from tensorflow.keras import *

In [2]:
import urllib.request
import zipfile
url = "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip"
file_name = "horse-or-human.zip"
training_dir = 'horse-or-human/training/'
urllib.request.urlretrieve(url, file_name)

zip_ref = zipfile.ZipFile(file_name, 'r')
zip_ref.extractall(training_dir)
zip_ref.close()

In [3]:
validation_url = "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip"

validation_file_name = "validation-horse-or-human.zip"
validation_dir = 'horse-or-human/validation/'

urllib.request.urlretrieve(validation_url, validation_file_name)
zip_ref = zipfile.ZipFile(validation_file_name, 'r')
zip_ref.extractall(validation_dir)
zip_ref.close()

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

training_datagen = tf.keras.preprocessing.image.ImageDataGenerator(1./255)
validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(1./255)

training_gen = training_datagen.flow_from_directory(
    training_dir, 
    target_size = (300, 300),
    batch_size = 32,
    class_mode = 'binary')
    
#class_mode would be set to 'categorical' if it was a multi-classification problem

    #train_datagen = ImageDataGenerator(
    #rescale=1./255,
    #rotation_range=40,
    #width_shift_range=0.2,
    #height_shift_range=0.2,
    #shear_range=0.2,
    #zoom_range=0.2,
    #horizontal_flip=True,
    #fill_mode='nearest'
    )#

validation_gen = validation_datagen.flow_from_directory(
    validation_dir,
    target_size = (300, 300),
    batch_size = 32, 
    class_mode = 'binary')

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation = tf.nn.relu, input_shape = (300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2), 
    tf.keras.layers.Conv2D(64, (3, 3), activation = tf.nn.relu),
    
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation = tf.nn.relu),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation = tf.nn.relu),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(64, (3,3), activation = tf.nn.relu),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation = tf.nn.relu),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation = tf.nn.relu),
    tf.keras.layers.Dense(10, activation = tf.nn.softmax)
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 298, 298, 64)      1792      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 149, 149, 64)      0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 147, 147, 64)      36928     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 73, 73, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 71, 71, 32)        18464     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 35, 35, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 33, 33, 64)        1

In [10]:
model.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(),
optimizer = tf.keras.optimizers.RMSprop(lr = 0.001), metrics = ['accuracy']) #set to categorical crossentropy for multiclassification

In [ ]:
import tensorflow as tf 

class myCallback1(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.95):
            print("/nReached 95% accuracy so stopping training!")
            self.model.stop_training = True

callbacks1 =  myCallback1()

class myCallback2(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs = {}):
        if(logs.get('accuracy')>0.99):
            print("/nReached 99% accuracy, therefore, stopping training!")
            self.model.stop_training = True

callbacks2 = myCallback2()

In [ ]:
epochs = 15

model.fit_generator(training_gen, validation_gen, epoch = epochs, callbacks=[callbacks2])

In [ ]:
img = image.load_img(img_path, target_size = (300, 300))
x = image.img_to_array(img)
x = np.expand_dims(x, axis = 0)

image_tensor = np.vstack([x])
class = model.predict(image_tensor)

if classes[0]>0.5:
    print(fn + " is a human")
    else:
    print(fn + " is a horse")